<a href="https://colab.research.google.com/github/lmcanavals/pcd/blob/master/9093_matmul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install golang-go

In [ ]:
%%file matmul.go
package main

import (
    "fmt"
)

var end chan bool

func source(row []float64, south chan float64) {
    for _, val := range row {
        south<- val
    }
    close(south)
}

func zero(n int, west chan float64) {
   for i := 0; i < n; i++ {
       west<- 0.0
   } 
   close(west)
}

func sink(north chan float64) {
    for _ = range north {
        
    }
}

func result(c [][]float64, i int, east chan float64) {
    j := 0
    for val := range east {
      c[i][j] = val
      j++
    }
    end<- true
}

func mult(firstElement float64, north, east, south, west chan float64) {
    for secondElement := range north {
        sum := <- east
        sum += firstElement * secondElement
        south <- secondElement
        west <- sum
    }
    close(south)
    close(west)
}

func main() {
    a := [][]float64{{1, 2, 3},
                     {4, 5, 6},
                     {7, 8, 9}}
    b := [][]float64{{1, 0, 2},
                     {0, 1, 2},
                     {1, 0, 0}}
    rowsa := len(a)
    colsa := len(a[0])
    colsb := len(b[0])

    end = make(chan bool)

    c := make([][]float64, rowsa)
    for i := range c {
        c[i] = make([]float64, colsb)
    }

    // creating channels
    h := make([][]chan float64, rowsa)
    for i := range h {
        h[i] = make([]chan float64, colsa + 1)
        for j := range h[i] {
            h[i][j] = make(chan float64)
        }
    }
    v := make([][]chan float64, rowsa + 1)
    for i := range v {
        v[i] = make([]chan float64, colsa)
        for j := range v[i] {
            v[i][j] = make(chan float64)
        }
    }

    for i := range b {
        go source(b[i], v[0][i])
        go sink(v[rowsa][i])
    }
    for i := range a {
        go zero(colsb, h[i][colsa])
        go result(c, i, h[i][0])
    }
    for i := range a {
        for j := range b {
            go mult(a[i][j], v[i][j], h[i][j + 1], v[i + 1][j], h[i][j])
        }
    }
    for _ = range a {
      <- end
    }
    fmt.Println(c)
}

Overwriting matmul.go


In [ ]:
!go run matmul.go

[[4 2 6] [10 5 18] [16 8 30]]


In [ ]:
import numpy as np
a = np.array([i for i in range(1, 10)]).reshape((3, 3))
b = np.array([1,0, 2,0, 1, 2, 1 ,0, 0]).reshape((3, 3))
print(np.matmul(a, b))

[[ 4  2  6]
 [10  5 18]
 [16  8 30]]
